<a href="https://colab.research.google.com/github/anhvt00/PIPR/blob/master/d_script_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Check GPU hardware

In [ ]:
!nvidia-smi

Mon Jan 24 07:00:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    33W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Install and import libraries

In [ ]:
!pip install dscript
import sys
import argparse
import h5py
import datetime
import subprocess as sp
import numpy as np
import gc
import pandas as pd
import gzip as gz
from tqdm import tqdm

import os
import matplotlib

matplotlib.use("Agg")
import matplotlib.pyplot as plt
from sklearn.metrics import (
    precision_recall_curve,
    average_precision_score,
    roc_curve,
    roc_auc_score,
)
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import IterableDataset, DataLoader
from sklearn.metrics import average_precision_score as average_precision

import dscript
from dscript.utils import PairedDataset, collate_paired_sequences
from dscript.models.embedding import (
    IdentityEmbed,
    FullyConnectedEmbed,
)
from dscript.models.contact import ContactCNN
from dscript.models.interaction import ModelInteraction


### Download data files

In [ ]:
!wget https://raw.githubusercontent.com/anhvt00/PIPR/master/data/Tunning-architecture-dataset/yeast_train.tsv
!wget https://raw.githubusercontent.com/anhvt00/PIPR/master/data/Tunning-architecture-dataset/yeast_test.tsv
!wget https://raw.githubusercontent.com/anhvt00/PIPR/master/data/Tunning-architecture-dataset/yeast.fasta

--2022-01-24 01:35:46--  https://raw.githubusercontent.com/anhvt00/PIPR/master/data/Tunning-architecture-dataset/yeast_train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 143200 (140K) [text/plain]
Saving to: ‘yeast_train.tsv’

yeast_train.tsv     100%[===================>] 139.84K  --.-KB/s    in 0.01s   

2022-01-24 01:35:46 (13.4 MB/s) - ‘yeast_train.tsv’ saved [143200/143200]

--2022-01-24 01:35:46--  https://raw.githubusercontent.com/anhvt00/PIPR/master/data/Tunning-architecture-dataset/yeast_test.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiti

### Embedding fasta sequences

In [ ]:
!dscript embed --seqs human_test_dict.fasta --outfile /content/human.h5 --device 0

# Using CUDA device 0 - Tesla P100-PCIE-16GB
# Loading Model...
# Loading Sequences...
100% 1545/1545 [00:00<00:00, 28559.72it/s]
# 1545 Sequences Loaded
# Approximate Storage Required (varies by average sequence length): ~12.36GB
# Storing to /content/human.h5...
100% 1545/1545 [04:43<00:00,  5.46it/s]


### Utility functions

In [ ]:
def add_args(parser):
    """
    Create parser for command line utility.

    :meta private:
    """

    data_grp = parser.add_argument_group("Data")
    proj_grp = parser.add_argument_group("Projection Module")
    contact_grp = parser.add_argument_group("Contact Module")
    inter_grp = parser.add_argument_group("Interaction Module")
    train_grp = parser.add_argument_group("Training")
    misc_grp = parser.add_argument_group("Output and Device")

    # Data
    data_grp.add_argument("--train", help="Training data", required=True)
    data_grp.add_argument("--val", help="Validation data", required=True)
    data_grp.add_argument("--embedding", help="h5 file with embedded sequences", required=True)
    data_grp.add_argument(
        "--no-augment",
        action="store_false",
        dest='augment',
        help="Set flag to not augment data by adding (B A) for all pairs (A B)",
    )

    # Embedding model
    proj_grp.add_argument(
        "--projection-dim",
        type=int,
        default=100,
        help="Dimension of embedding projection layer (default: 100)",
    )
    proj_grp.add_argument(
        "--dropout-p",
        type=float,
        default=0.5,
        help="Parameter p for embedding dropout layer (default: 0.5)",
    )

    # Contact model
    contact_grp.add_argument(
        "--hidden-dim",
        type=int,
        default=50,
        help="Number of hidden units for comparison layer in contact prediction (default: 50)",
    )
    contact_grp.add_argument(
        "--kernel-width",
        type=int,
        default=7,
        help="Width of convolutional filter for contact prediction (default: 7)",
    )

    # Interaction Model
    inter_grp.add_argument(
        "--no-w",
        action="store_false",
        dest='use_w',
        help="Don't use weight matrix in interaction prediction model",
    )
    inter_grp.add_argument(
        "--pool-width",
        type=int,
        default=9,
        help="Size of max-pool in interaction model (default: 9)",
    )

    # Training
    train_grp.add_argument(
        "--negative-ratio",
        type=int,
        default=10,
        help="Number of negative training samples for each positive training sample (default: 10)",
    )
    train_grp.add_argument(
        "--epoch-scale",
        type=int,
        default=1,
        help="Report heldout performance every this many epochs (default: 1)",
    )
    train_grp.add_argument("--num-epochs", type=int, default=10, help="Number of epochs (default: 10)")
    train_grp.add_argument("--batch-size", type=int, default=25, help="Minibatch size (default: 25)")
    train_grp.add_argument("--weight-decay", type=float, default=0, help="L2 regularization (default: 0)")
    train_grp.add_argument("--lr", type=float, default=0.001, help="Learning rate (default: 0.001)")
    train_grp.add_argument(
        "--lambda",
        dest="lambda_",
        type=float,
        default=0.35,
        help="Weight on the similarity objective (default: 0.35)",
    )

    # Output
    misc_grp.add_argument("-o", "--outfile", help="Output file path (default: stdout)")
    misc_grp.add_argument("--save-prefix", help="Path prefix for saving models")
    misc_grp.add_argument("-d", "--device", type=int, default=-1, help="Compute device to use")
    misc_grp.add_argument("--checkpoint", help="Checkpoint model to start training from")

    return parser


def predict_interaction(model, n0, n1, tensors, use_cuda):
    """
    Predict whether a list of protein pairs will interact.

    :param model: Model to be trained
    :type model: dscript.models.interaction.ModelInteraction
    :param n0: First protein names
    :type n0: list[str]
    :param n1: Second protein names
    :type n1: list[str]
    :param tensors: Dictionary of protein names to embeddings
    :type tensors: dict[str, torch.Tensor]
    :param use_cuda: Whether to use GPU
    :type use_cuda: bool
    """

    b = len(n0)

    p_hat = []
    for i in range(b):
        z_a = tensors[n0[i]]
        z_b = tensors[n1[i]]
        if use_cuda:
            z_a = z_a.cuda()
            z_b = z_b.cuda()

        p_hat.append(model.predict(z_a, z_b))
    p_hat = torch.stack(p_hat, 0)
    return p_hat


def predict_cmap_interaction(model, n0, n1, tensors, use_cuda):
    """
    Predict whether a list of protein pairs will interact, as well as their contact map.

    :param model: Model to be trained
    :type model: dscript.models.interaction.ModelInteraction
    :param n0: First protein names
    :type n0: list[str]
    :param n1: Second protein names
    :type n1: list[str]
    :param tensors: Dictionary of protein names to embeddings
    :type tensors: dict[str, torch.Tensor]
    :param use_cuda: Whether to use GPU
    :type use_cuda: bool
    """

    b = len(n0)

    p_hat = []
    c_map_mag = []
    for i in range(b):
        z_a = tensors[n0[i]]
        z_b = tensors[n1[i]]
        if use_cuda:
            z_a = z_a.cuda()
            z_b = z_b.cuda()

        cm, ph = model.map_predict(z_a, z_b)
        p_hat.append(ph)
        c_map_mag.append(torch.mean(cm))
    p_hat = torch.stack(p_hat, 0)
    c_map_mag = torch.stack(c_map_mag, 0)
    return c_map_mag, p_hat


def interaction_grad(model, n0, n1, y, tensors, use_cuda, weight=0.35):
    """
    Compute gradient and backpropagate loss for a batch.

    :param model: Model to be trained
    :type model: dscript.models.interaction.ModelInteraction
    :param n0: First protein names
    :type n0: list[str]
    :param n1: Second protein names
    :type n1: list[str]
    :param y: Interaction labels
    :type y: torch.Tensor
    :param tensors: Dictionary of protein names to embeddings
    :type tensors: dict[str, torch.Tensor]
    :param use_cuda: Whether to use GPU
    :type use_cuda: bool
    :param weight: Weight on the contact map magnitude objective. BCE loss is :math:`1 - \\text{weight}`.
    :type weight: float

    :return: (Loss, number correct, mean square error, batch size)
    :rtype: (torch.Tensor, int, torch.Tensor, int)
    """

    c_map_mag, p_hat = predict_cmap_interaction(model, n0, n1, tensors, use_cuda)
    if use_cuda:
        y = y.cuda()
    y = Variable(y)

    bce_loss = F.binary_cross_entropy(p_hat.float(), y.float())
    cmap_loss = torch.mean(c_map_mag)
    loss = (weight * bce_loss) + ((1 - weight) * cmap_loss)
    b = len(p_hat)

    # backprop loss
    loss.backward()

    if use_cuda:
        y = y.cpu()
        p_hat = p_hat.cpu()

    with torch.no_grad():
        guess_cutoff = 0.5
        p_hat = p_hat.float()
        p_guess = (guess_cutoff * torch.ones(b) < p_hat).float()
        y = y.float()
        correct = torch.sum(p_guess == y).item()
        mse = torch.mean((y.float() - p_hat) ** 2).item()
    return loss, correct, mse, b


def interaction_eval(model, test_iterator, tensors, use_cuda):
    """
    Evaluate test data set performance.

    :param model: Model to be trained
    :type model: dscript.models.interaction.ModelInteraction
    :param test_iterator: Test data iterator
    :type test_iterator: torch.utils.data.DataLoader
    :param tensors: Dictionary of protein names to embeddings
    :type tensors: dict[str, torch.Tensor]
    :param use_cuda: Whether to use GPU
    :type use_cuda: bool

    :return: (Loss, number correct, mean square error, precision, recall, F1 Score, AUPR)
    :rtype: (torch.Tensor, int, torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor)
    """
    p_hat = []
    true_y = []

    for n0, n1, y in test_iterator:
        p_hat.append(predict_interaction(model, n0, n1, tensors, use_cuda))
        true_y.append(y)

    y = torch.cat(true_y, 0)
    p_hat = torch.cat(p_hat, 0)

    if use_cuda:
        y.cuda()
        p_hat = torch.Tensor([x.cuda() for x in p_hat])
        p_hat.cuda()

    loss = F.binary_cross_entropy(p_hat.float(), y.float()).item()
    b = len(y)

    with torch.no_grad():
        guess_cutoff = torch.Tensor([0.5]).float()
        p_hat = p_hat.float()
        y = y.float()
        p_guess = (guess_cutoff * torch.ones(b) < p_hat).float()
        correct = torch.sum(p_guess == y).item()
        mse = torch.mean((y.float() - p_hat) ** 2).item()

        tp = torch.sum(y * p_hat).item()
        pr = tp / torch.sum(p_hat).item()
        re = tp / torch.sum(y).item()
        f1 = 2 * pr * re / (pr + re)

    y = y.cpu().numpy()
    p_hat = p_hat.data.cpu().numpy()

    aupr = average_precision(y, p_hat)
    return loss, correct, mse, pr, re, f1, aupr


def plot_eval_predictions(labels, predictions, path="figure"):
    """
    Plot histogram of positive and negative predictions, precision-recall curve, and receiver operating characteristic curve.

    :param y: Labels
    :type y: np.ndarray
    :param phat: Predicted probabilities
    :type phat: np.ndarray
    :param path: File prefix for plots to be saved to [default: figure]
    :type path: str
    """

    pos_phat = predictions[labels == 1]
    neg_phat = predictions[labels == 0]

    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.suptitle("Distribution of Predictions")
    ax1.hist(pos_phat)
    ax1.set_xlim(0, 1)
    ax1.set_title("Positive")
    ax1.set_xlabel("p-hat")
    ax2.hist(neg_phat)
    ax2.set_xlim(0, 1)
    ax2.set_title("Negative")
    ax2.set_xlabel("p-hat")
    plt.savefig(path + ".phat_dist.png")
    plt.close()

    precision, recall, pr_thresh = precision_recall_curve(labels, predictions)
    aupr = average_precision_score(labels, predictions)
    print("AUPR:", aupr)

    plt.step(recall, precision, color="b", alpha=0.2, where="post")
    plt.fill_between(recall, precision, step="post", alpha=0.2, color="b")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title("Precision-Recall (AUPR: {:.3})".format(aupr))
    plt.savefig(path + ".aupr.png")
    plt.close()

    fpr, tpr, roc_thresh = roc_curve(labels, predictions)
    auroc = roc_auc_score(labels, predictions)
    print("AUROC:", auroc)

    plt.step(fpr, tpr, color="b", alpha=0.2, where="post")
    plt.fill_between(fpr, tpr, step="post", alpha=0.2, color="b")
    plt.xlabel("FPR")
    plt.ylabel("TPR")
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title("Receiver Operating Characteristic (AUROC: {:.3})".format(auroc))
    plt.savefig(path + ".auroc.png")
    plt.close()

def get_state_dict(version="human_v1", verbose=True):
    """
    Download a pre-trained model if not already exists on local device.

    :param version: Version of trained model to download [default: human_1]
    :type version: str
    :param verbose: Print model download status on stdout [default: True]
    :type verbose: bool
    :return: Path to state dictionary for pre-trained language model
    :rtype: str
    """
    state_dict_basename = f"dscript_{version}.pt"
    state_dict_fullname = f"/content/{state_dict_basename}"
    state_dict_url = f"http://cb.csail.mit.edu/cb/dscript/data/models/{state_dict_basename}"
    if not os.path.exists(state_dict_fullname):
        try:
            import urllib.request
            import shutil
            if verbose: print(f"Downloading model {version} from {state_dict_url}...")
            with urllib.request.urlopen(state_dict_url) as response, open(state_dict_fullname, 'wb') as out_file:
                shutil.copyfileobj(response, out_file)
        except Exception as e:
            print("Unable to download model - {}".format(e))
            sys.exit(1)
    return state_dict_fullname

### Prepare dataloader object for train and test sets

In [ ]:
# Specify output file
output = sys.stdout

print(f'# Called as: {" ".join(sys.argv)}', file=output)
if output is not sys.stdout:
    print(f'Called as: {" ".join(sys.argv)}')

# Set device for loading data
device = 0
use_cuda = (device >= 0) and torch.cuda.is_available()
if use_cuda:
    torch.cuda.set_device(device)
    print(
        f"# Using CUDA device {device} - {torch.cuda.get_device_name(device)}",
        file=output,
    )
else:
    print("# Using CPU", file=output)
    device = "cpu"

# prepare for loading data
batch_size = 1
train_fi = 'human_train.tsv'
test_fi = 'human_test.tsv'
augment = True
embedding_h5 = 'human.h5'
h5fi = h5py.File(embedding_h5, "r")

print(f"# Loading training pairs from {train_fi}...", file=output)
output.flush()

# read train index pairs
train_df = pd.read_csv(train_fi, sep="\t", header=None)
if augment:
    train_n0 = pd.concat((train_df[0], train_df[1]), axis=0).reset_index(drop=True)
    train_n1 = pd.concat((train_df[1], train_df[0]), axis=0).reset_index(drop=True)
    train_y = torch.from_numpy(pd.concat((train_df[2], train_df[2])).values)
else:
    train_n0, train_n1 = train_df[0], train_df[1]
    train_y = torch.from_numpy(train_df[2].values)

print(f"# Loading testing pairs from {test_fi}...", file=output)
output.flush()

# read test index pairs
test_df = pd.read_csv(test_fi, sep="\t", header=None)
test_n0, test_n1 = test_df[0], test_df[1]
test_y = torch.from_numpy(test_df[2].values)
output.flush()

# create train index pairs dataloader
train_pairs = PairedDataset(train_n0, train_n1, train_y)
pairs_train_iterator = torch.utils.data.DataLoader(
    train_pairs,
    batch_size=batch_size,
    collate_fn=collate_paired_sequences,
    shuffle=True,
)

# create test index pairs dataloader
test_pairs = PairedDataset(test_n0, test_n1, test_y)
pairs_test_iterator = torch.utils.data.DataLoader(
    test_pairs,
    batch_size=batch_size,
    collate_fn=collate_paired_sequences,
    shuffle=True,
)

output.flush()

# load embeddings of sequences into cpu ram 
print(f"# Loading embeddings", file=output)
tensors = {}
all_proteins = set(train_n0).union(set(train_n1)).union(set(test_n0)).union(set(test_n1))
for prot_name in tqdm(all_proteins):
    tensors[prot_name] = torch.from_numpy(h5fi[prot_name][:, :])

# Called as: /usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-ac06099f-20e2-438a-8dc6-1956db621a7d.json
# Using CUDA device 0 - Tesla P100-PCIE-16GB
# Loading training pairs from human_train.tsv...
# Loading testing pairs from human_test.tsv...
# Loading embeddings


100%|██████████| 1038/1038 [00:24<00:00, 42.38it/s]


### Define architecture of d-script model

In [ ]:
use_cuda = (0 > -1) and torch.cuda.is_available()
checkpoint = 'human_v1.sav'
if checkpoint is None:
    # Create projection module
    projection_dim = 100
    dropout_p = .5
    embedding = FullyConnectedEmbed(6165, projection_dim, dropout=dropout_p)
    print("# Initializing embedding model with:", file=output)
    print(f"\tprojection_dim: {projection_dim}", file=output)
    print(f"\tdropout_p: {dropout_p}", file=output)

    # Create contact module
    hidden_dim = 50
    kernel_width = 7
    print("# Initializing contact model with:", file=output)
    print(f"\thidden_dim: {hidden_dim}", file=output)
    print(f"\tkernel_width: {kernel_width}", file=output)

    contact = ContactCNN(projection_dim, hidden_dim, kernel_width)

    # Create the full model including projection module and contact module
    use_W = True
    pool_width = 9
    print("# Initializing interaction model with:", file=output)
    print(f"\tpool_width: {pool_width}", file=output)
    print(f"\tuse_w: {use_W}", file=output)
    model = ModelInteraction(embedding, contact, use_W=use_W, pool_size=pool_width)

    print(model, file=output)

else:
    print("# Loading model from checkpoint {}".format(checkpoint), file=output)
    model = torch.load(checkpoint)
    model.use_cuda = use_cuda

if use_cuda:
    model = model.cuda()

# Loading model from checkpoint human_v1.sav


### Hyperparameters for training

In [ ]:
lr = .001
wd = .0
num_epochs = 20
batch_size = 1
report_steps = 1
inter_weight = .35
cmap_weight = 1 - inter_weight
digits = int(np.floor(np.log10(num_epochs))) + 1
save_prefix = None
if save_prefix is None:
    save_prefix = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")

params = [p for p in model.parameters() if p.requires_grad]
optim = torch.optim.Adam(params, lr=lr, weight_decay=wd)

print(f'# Using save prefix "{save_prefix}"', file=output)
print(f"# Training with Adam: lr={lr}, weight_decay={wd}", file=output)
print(f"\tnum_epochs: {num_epochs}", file=output)
print(f"\tepoch_scale: {report_steps}", file=output)
print(f"\tbatch_size: {batch_size}", file=output)
print(f"\tinteraction weight: {inter_weight}", file=output)
print(f"\tcontact map weight: {cmap_weight}", file=output)
output.flush()

batch_report_fmt = "# [{}/{}] training {:.1%}: Loss={:.6}, Accuracy={:.3%}, MSE={:.6}"
epoch_report_fmt = "# Finished Epoch {}/{}: Loss={:.6}, Accuracy={:.3%}, MSE={:.6}, Precision={:.6}, Recall={:.6}, F1={:.6}, AUPR={:.6}"

# Using save prefix "2022-01-24-07-42"
# Training with Adam: lr=0.001, weight_decay=0.0
	num_epochs: 20
	epoch_scale: 1
	batch_size: 1
	interaction weight: 0.35
	contact map weight: 0.65


### Train on training dataset and evaluation on test set

In [ ]:
N = len(pairs_train_iterator) * batch_size
for epoch in range(num_epochs):

    model.train()

    n = 0
    loss_accum = 0
    acc_accum = 0
    mse_accum = 0

    # Train batches
    for (z0, z1, y) in tqdm(pairs_train_iterator, desc=f"Epoch {epoch+1}/{num_epochs}",total=len(pairs_train_iterator)):

        loss, correct, mse, b = interaction_grad(model, z0, z1, y, tensors, use_cuda, weight=inter_weight)

        n += b
        delta = b * (loss - loss_accum)
        loss_accum += delta / n

        delta = correct - b * acc_accum
        acc_accum += delta / n

        delta = b * (mse - mse_accum)
        mse_accum += delta / n

        report = (n - b) // 100 < n // 100

        optim.step()
        optim.zero_grad()
        model.clip()

        if report:
            tokens = [
                epoch + 1,
                num_epochs,
                n / N,
                loss_accum,
                acc_accum,
                mse_accum,
            ]
            if output is not sys.stdout:
                print(batch_report_fmt.format(*tokens), file=output)
                output.flush()

    if (epoch + 1) % report_steps == 0:
        model.eval()

        with torch.no_grad():

            (
                inter_loss,
                inter_correct,
                inter_mse,
                inter_pr,
                inter_re,
                inter_f1,
                inter_aupr,
            ) = interaction_eval(model, pairs_test_iterator, tensors, use_cuda)
            tokens = [
                epoch + 1,
                num_epochs,
                inter_loss,
                inter_correct / (len(pairs_test_iterator) * batch_size),
                inter_mse,
                inter_pr,
                inter_re,
                inter_f1,
                inter_aupr,
            ]
            print(epoch_report_fmt.format(*tokens), file=output)
            output.flush()

        # Save the model
        if save_prefix is not None:
            save_path = save_prefix + "_epoch" + str(epoch + 1).zfill(digits) + ".sav"
            print(f"# Saving model to {save_path}", file=output)
            model.cpu()
            torch.save(model, save_path)
            if use_cuda:
                model.cuda()

    # collect variable by garbage collector and empty cache in cuda
    output.flush()

if save_prefix is not None:
    save_path = save_prefix + "_final.sav"
    print(f"# Saving final model to {save_path}", file=output)
    model.cpu()
    torch.save(model, save_path)
    if use_cuda:
        model.cuda()

output.close()

Epoch 1/20: 100%|██████████| 880/880 [00:55<00:00, 15.81it/s]


# Finished Epoch 1/20: Loss=0.547597, Accuracy=90.090%, MSE=0.0982272, Precision=0.123681, Recall=0.00446679, F1=0.00862219, AUPR=0.389835
# Saving model to 2022-01-24-07-42_epoch01.sav


Epoch 2/20: 100%|██████████| 880/880 [00:55<00:00, 15.83it/s]


# Finished Epoch 2/20: Loss=0.56626, Accuracy=90.090%, MSE=0.0984395, Precision=0.0943053, Recall=0.00340927, F1=0.00658064, AUPR=0.156991
# Saving model to 2022-01-24-07-42_epoch02.sav


Epoch 3/20: 100%|██████████| 880/880 [00:55<00:00, 15.83it/s]


...

# Finished Epoch 18/20: Loss=0.538839, Accuracy=90.991%, MSE=0.0892661, Precision=0.615871, Recall=0.180248, F1=0.278877, AUPR=0.413039
# Saving model to 2022-01-24-07-42_epoch18.sav


Epoch 19/20: 100%|██████████| 880/880 [00:55<00:00, 15.86it/s]


# Finished Epoch 19/20: Loss=0.550474, Accuracy=90.991%, MSE=0.0912651, Precision=0.651481, Recall=0.0553717, F1=0.102068, AUPR=0.45325
# Saving model to 2022-01-24-07-42_epoch19.sav


Epoch 20/20: 100%|██████████| 880/880 [00:55<00:00, 15.85it/s]


# Finished Epoch 20/20: Loss=0.461964, Accuracy=90.090%, MSE=0.0902703, Precision=0.570685, Recall=0.120456, F1=0.198924, AUPR=0.709059
# Saving model to 2022-01-24-07-42_epoch20.sav


### Evaluation of pretrained model on new data 

In [ ]:
# Set Device
device = 0
use_cuda = (device >= 0) and torch.cuda.is_available()
if use_cuda:
    torch.cuda.set_device(device)
    print(f"# Using CUDA device {device} - {torch.cuda.get_device_name(device)}")
else:
    print("# Using CPU")

# Load Model
model_path = 'human_v1.sav'
if use_cuda:
    model = torch.load(model_path).cuda()
else:
    model = torch.load(model_path).cpu()
    model.use_cuda = False

embeddingPath = 'human.h5'
h5fi = h5py.File(embeddingPath, "r")

# Load Pairs
test_fi = 'human_test_100_pos.tsv'
test_df = pd.read_csv(test_fi, sep="\t", header=None)

outPath = None
if outPath is None:
    outPath = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")
else:
    outPath = args.outfile
outFile = open(outPath + ".predictions.tsv", "w+")

allProteins = set(test_df[0]).union(test_df[1])

seqEmbDict = {}
for i in tqdm(allProteins, desc="Loading embeddings"):
    seqEmbDict[i] = torch.from_numpy(h5fi[i][:]).float()

model.eval()
with torch.no_grad():
    phats = []
    labels = []
    for _, (n0, n1, label) in tqdm(test_df.iterrows(), total=len(test_df), desc="Predicting pairs"):
        try:
            p0 = seqEmbDict[n0]
            p1 = seqEmbDict[n1]
            if use_cuda:
                p0 = p0.cuda()
                p1 = p1.cuda()

            pred = model.predict(p0, p1).item()
            phats.append(pred)
            labels.append(label)
            print("{}\t{}\t{}\t{:.5}".format(n0, n1, label, pred), file=outFile)
        except Exception as e:
            sys.stderr.write("{} x {} - {}".format(n0, n1, e))

phats = np.array(phats)
labels = np.array(labels)
plot_eval_predictions(labels, phats, outPath)

outFile.close()
h5fi.close()


# Using CUDA device 0 - Tesla P100-PCIE-16GB


Predicting pairs: 100%|██████████| 550/550 [00:07<00:00, 74.60it/s]


AUPR: 0.6004142210493548
AUROC: 0.8289000000000001


In [ ]:
!cp human_test_100.tsv human_test.tsv